### Question1: most important features for predicting rental prices

In our classification model, the four most important features for predicting rental prices are as follows:


1. Bedrooms: The number of bedrooms is a significant determinant of rental prices, as it directly correlates with the capacity and comfort of the living space. 
Properties with more bedrooms typically cater to larger households, making them more desirable and, consequently, commanding higher rental rates.


2. Postcode: The postcode serves as an indicator of the property's location, which significantly influences rental values. 
Areas with desirable postcodes often offer better access to amenities, transportation, and schools, 
contributing to higher demand and increased rental prices.


3. Bathrooms: The quantity of bathrooms is another crucial factor impacting rental pricing.
A higher number of bathrooms enhances convenience for tenants, 
especially in larger households, and is often seen as a desirable feature that can justify a premium in rental costs.


4. Parking Spaces: The availability of parking spaces is a vital consideration in many regions, 
particularly in urban settings where parking can be scarce. 
Properties that offer designated parking tend to attract tenants who prioritize this convenience, 
thereby increasing the potential rental value.

By pearson correlation, the most significant features for predicting rental prices are bedrooms numbers and bathroom numbers.\
There is no significant correlation between distance to the closest train station and rental prices

<img src="../plots/pearson_corr.png" alt="pearson" />

With Spearman's correlation, bedroom number is still the most significant.\
The correlation is almost zero between distance to the closest train station and rental price.

<img src="../plots/spearmans_corr.png" alt="spearsmans" />

By using Anova, there are sufficent evidences to the effect of suburb and property type on rental price.
Suburb have a 0.000172 p-vlaue, and property type have the p-value equal to 1.878596e-41.

### Question2: top 10 suburbs with the highest predicted growth rate

**Top 10 suburbs with highest predicted growth rate for all properties**
1. Caulfield:&nbsp;0.605755
2. CBD-St Kilda Rd:&nbsp;0.456993
3. Richmond-Burnley:&nbsp;0.259125
4. Carlton-Parkville:&nbsp;0.221304
5. Thomastown-Lalor:&nbsp;0.216575
6. Malvern East:&nbsp;0.215734
7. Hawthorn:&nbsp;0.211035
8. Clayton:&nbsp;0.208304
9. East St Kilda:&nbsp;0.205950
10. Springvale:&nbsp;0.203281